# <center>Machine Learning</center>

## Classification MNIST à l'aide d'un réseau de neurones dense
### Clément AUBEUF

----

<a
  target="_blank" href="https://colab.research.google.com/drive/10w9E1whCPMZZposk2vru9cYNEA3CavuG"> 
  <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> 
  Ouvrir dans Google Colab
</a>

----

### Importation de bibliothèques

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

import math
import numpy as np
import matplotlib.pyplot as plt

### Importation de la banque de données MNIST fashion

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
class_names = metadata.features['label'].names

# On affiche les étiquettes des vêtements disponibles
print("Labels des vêtements : {}".format(class_names))

On importe la banque de données MNIST fashion (les images de vêtements) ainsi que leurs étiquettes (pullover, shirt etc ...)

### Séparation des données

La majeure partie servira à entrainer le modèle, tandis que le reste des données servira de test à notre modèle

In [ ]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

print("Nombre d'exemples pour l'entrainement : {}".format(num_train_examples))
print("Nombre d'exemples pour le test final : {}".format(num_test_examples))

### Normalisation des données

Les images sont actuellement codées avec des valeurs comprises entre 0 et 255. Pour améliorer l'apprentissage du modèle, on préfèrera les codées dans une intervalle [0,1].

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# On applique la fonction de normalisation aux images d'entrainements et de test
train_dataset =  train_dataset.map(normalize)
test_dataset  =  test_dataset.map(normalize)

# Et on place les images dans le cache plutôt que sur le disque (ce qui améliore la rapidité d'entrainement du modèle)
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

### Visualisation des données actuelles

In [ ]:
# Pour afficher une image, on doit retirer les couleurs : on utilise reshape afin de retirer la 3e dimension
# On passe alors d'une image dimension/dimension/couleur à une image dimension/dimension
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

# Et on affiche l'image
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.grid(False)
plt.show()

On peut appliquer ce code de manière récursive : celà permet d'afficher plusieures images.

In [ ]:
# On dimensionne le tableau qui contiendra nos images
plt.figure(figsize=(10,10))

# Et on applique le code utilisé précédemment
for i, (image, label) in enumerate(test_dataset.take(25)):
    image = image.numpy().reshape((28,28))
    # On dimensionne les images afin de les rendre plus petites pour en afficher plus
    plt.subplot(5,5,i+1)
    # On retire les valeurs des axes X et Y (la dimension des images, qui ne nous interesse pas ici)
    plt.xticks([])
    plt.yticks([])
    # On retire la grille
    plt.grid(False)
    # On affiche l'image
    plt.imshow(image, cmap=plt.cm.binary)
    # Et on ajoute l'étiquette correspondant au vêtement
    plt.xlabel(class_names[label])

plt.show()

### Création du modèle

In [ ]:
# On crée notre modèle en utilisant un réseau de neurones dense
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

On distingue ici trois layers : <br>
- un layer d'entrée (layers.Flatten) qui transforme l'image 2D (dimensions et couleur) en image 1D (dimensions) 
- un layer caché composé de 128 neurones et utilisant la méthode relu 
- un layer de sortie composé de 10 noeuds (qui correspondent aux 10 types de vêtements)

In [ ]:
# Notre modèle a encore besoin de quelques paramètres avant d'être entrainé : 
# on définit la méthode d'optimisation, la fonction de perte et le paramètre sur lequel le modèle doit s'entrainer.
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# metrics=['accuracy'] signifie que le modèle classera ces résultats en fonction de sa précision (est ce que le vêtement est bien ou mal nommé ?)

### Entrainement du modèle

In [ ]:
# On définit le comportement que le programme doit suivre pour son apprentissage
lots_img = 32
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(lots_img)
test_dataset = test_dataset.cache().batch(lots_img)

# .cache = place les données dans le cache
# .repeat = le modèle répètera cette opération à l'infini
# .shuffle = mélange les données (en fonction du nombre de données)
# .batch = le modèle apprend par lot de 32 images

In [ ]:
# Et on entraine le modèle
model.fit(train_dataset, epochs=3, steps_per_epoch=math.ceil(num_train_examples/lots_img))

### Précision du modèle

In [ ]:
# On peut tester la précision du modèle sur les données de test
# Afin de gagner du temps, nous n'utiliserons qu'une fraction de ces données (ici 10.000/50, soit 200 données)
test_loss, test_accuracy = model.evaluate(test_dataset, steps=math.ceil(num_test_examples/50))

print('Précision du modèle :', int(round(test_accuracy, 2)*100),'%')

### Application du modèle

Appliquons notre modèle à un lot de vêtements :

In [ ]:
# On applique le modèle à un lot d'images : take(1)
# Ce lot est constitué de 32 images (définit précédemment)
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

----

On teste le vêtement à l'index 20 :

In [ ]:
print(predictions[20])

On obtient un tableau contenant la probabilité de chaque étiquette (tshirt, pull, shoes ...) pour ce vêtement.

In [ ]:
# On isole la probabilité la plus grande
print('Probabilité la plus haute : étiquette de vêtement numéro', np.argmax(predictions[20]))

# et on vérifie la réponse correcte
print('Réponse correcte : étiquette de vêtement numéro', test_labels[20])

# Ce qui correspond à
print('Vêtement détecté :', class_names[0])

Le vêtement à l'index 20 est donc bien un T-shirt : notre modèle ne s'est pas trompé !

### Affichage des résultats

In [ ]:
# On définit la fonction permettant d'afficher le vêtement en utilisant un paramètre i (qui correspond à l'index du vêtement)
# Note : cette fonction est très similaire à celle utilisée en début de page pour afficher le vêtement
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img[...,0], cmap=plt.cm.binary)
  predicted_label = np.argmax(predictions_array)

  # En fonction de la prédiction, le nom du vêtement s'affiche en vert (correct) ou en rouge
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'
  
  # Et on ajoute l'étiquette du vêtement
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# On crée une seconde fonction qui permet d'afficher les probabilités sous forme de graphique, toujours en fonction de i
def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  # Toutes les prédictions apparaitront en vert clair
  thisplot = plt.bar(range(10), predictions_array, color="lightgreen")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
  
  # On superpose les barres du graph :
  # - si le résultat est bon, une barre verte foncée sera visible
  # - sinon, nous verrons une barre verte (le bon résultat) et une barre rouge (le résultat prédit)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('green')

La mise en forme des résultats est définie, il ne reste plus qu'à tester !

----

Testons le vêtement à l'index 5.

In [ ]:
i = 5

# On affiche le graph
plt.figure(figsize=(15,3))
plt.subplot(1,2,1)

# l'image
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)

# on place les résultats sur le graphique
plot_value_array(i, predictions, test_labels)

# Et on affiche les catégories du graphique
_ = plt.xticks(range(10), class_names, rotation=45)

----

Et le vêtement index 20 ?

In [ ]:
i = 20

plt.figure(figsize=(15,3))
plt.subplot(1,2,1)

plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)

plot_value_array(i, predictions, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

----

Enfin, on peut afficher plusieurs résultats d'un seul coup :

In [ ]:
nbr_lignes = 3
nbr_col = 3
nbr_imgs = nbr_lignes * nbr_col

plt.figure(figsize=(16, 8))
for i in range(nbr_imgs):
  # On affiche l'image et sa prédiction
  plt.subplot(nbr_lignes, 2*nbr_col, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  # On affiche le graphique de prédictions
  plt.subplot(nbr_lignes, 2*nbr_col, 2*i+2)  
  plot_value_array(i, predictions, test_labels)